## Visualization of supported behavior model for cars and pedestrian

In [6]:
using AutomotivePOMDPs, AutoViz, AutomotiveDrivingModels
using Reel

In [13]:
cam = FitToContentCamera(0.)
rng = MersenneTwister(1);

## Intelligent Pedestrian

In [63]:
env = CrosswalkEnv()
env.obstacles = ConvexPolygon[] # no obstacles
pomdp = OCPOMDP(env=env); # to have convenient functions

In [64]:
scene = Scene()
ego = Vehicle(VehicleState(VecSE2(10., 0., 0.0), env.roadway, 6.), VehicleDef(), 1)
push!(scene, ego)
ped =  Vehicle(VehicleState(Frenet(env.crosswalk, 5.), env.roadway, 1.), AutomotivePOMDPs.PEDESTRIAN_DEF, 2)
push!(scene, ped)

models = Dict{Int64, DriverModel}()
models[ego.id] = IntelligentDriverModel(v_des=5.) # follow its own lane
models[ped.id] = IntelligentPedestrian(crosswalk=env.crosswalk, conflict_lanes=get_conflict_lanes(env.crosswalk, env.roadway))

nticks = 100
timestep = 0.1
rec = SceneRecord(nticks+1, timestep)
# execute the simulation
simulate!(rec, scene, env.roadway, models, nticks)
duration, fps, render_rec = animate_record(env, rec, sim_dt=timestep, cam=cam, overlays=SceneOverlay[IDOverlay()])
film = roll(render_rec, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmps1fbpw", 0x0000000000000065, 10.0, nothing)